In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import re
import json
import h2o

from datetime import datetime
from functools import lru_cache

from utils import get_data, Config

from tree import make_tree

In [4]:
def clean_url(url):
    # Set browser_id=1 (unknown) to not accidentally include them in our real data collection
    return re.sub(r"browser_id=(\d+)", "browser_id=1", url)

def make_clickable(url):
    # Clickable links for debugging
    url = clean_url(url)
    return f'<a href="{url}" target="_blank">{url}</a>'

In [5]:
@lru_cache(maxsize=None)
def id_to_browser(id):
    """Full browser name (os+version) from the ID"""
    return " ".join(df.loc[df["browser_id"] == id].iloc[0][["name", "os", "version", "automation_mode", "headless_mode"]].to_list())

## Load Data + Preprocessing

In [6]:
# Load all data (except unknown/manual testing)
initial_data = """
SELECT "Result".*, 
"Response".raw_header, "Response".status_code, "Response".label, "Response".resp_type,
"Browser".name, "Browser".version, "Browser".headless_mode, "Browser".os, "Browser".automation_mode, "Browser".add_info
FROM "Result"
JOIN "Response" ON "Result".response_id = "Response".id JOIN "Browser" ON "Result".browser_id = "Browser".id
WHERE "Browser".name != 'Unknown';
"""
df = get_data(Config(), initial_data)

Connecting to the PostgreSQL database...
Connection successful


In [7]:
# Create extra columns
df["outcome_str"] = df["outcome_value"].fillna("None").astype(str)
df["clean_url"] = df["full_url"].apply(clean_url)
df["browser"] = df["browser_id"].apply(id_to_browser)
df["org_origin"] = df["org_scheme"] + "://" + df["org_host"]
df["resp_origin"] = df["resp_scheme"] + "://" + df["resp_host"]

# Unify outcomes that are semantically the same (only the exact error string is different in different browsers)
# Firefox: {'error': 'object "TypeError: NetworkError when attempting to fetch resource."', 'headers': ''}
# Chromium: {'error': 'object "TypeError: Failed to fetch"', 'headers': ''}
# Safari: {'error': 'object "TypeError: Load failed"', 'headers': ''}
# ...
df["outcome_str"] = df["outcome_str"].replace("{'error': 'object \"TypeError: NetworkError when attempting to fetch resource.\"', 'headers': ''}", "{'error': 'object \"TypeError: Failed to fetch\"', 'headers': ''}")
df["outcome_str"] = df["outcome_str"].replace("{'error': 'object \"TypeError: Load failed\"', 'headers': ''}", "{'error': 'object \"TypeError: Failed to fetch\"', 'headers': ''}")

# For document referrer we do not want to know the exact resp_id and count
# We only want to know whether it is a origin or the full URl?
#df['outcome_str'] = df['outcome_str'].replace(r'resp_id=\d+', 'resp_id=<resp_id>', regex=True)
#df['outcome_str'] = df['outcome_str'].replace(r'count=\d+', 'count=<count>', regex=True)
df['outcome_str'] = df['outcome_str'].apply(lambda x: 'document.referrer: full_url' if 'responses.py?feature_group' in x else x)



In [8]:
df["outcome_str"].unique()

array(['window.originAgentCluster: true',
       'document.referrer: https://headers.websec.saarland/',
       "{'response.redirected': False}",
       "{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'load'}, 'window.crossOriginIsolated': False}",
       "{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'error', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}",
       "{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}",
       'No performance entry', "{'requestStart != 0': True}",
       'document.referrer: ',
       "{'error': 'null', 'headers': 'access-control-allow-origin,content-length,date,server,test,'}",
       "{'error': 'null', 'headers': 'content-length,'}",
       '{\'error\': \'object "TypeError: Failed to fetch"\', \

## Overview
- Collected data
- Timeouts
- Time taken

In [9]:
# Collected results per browser
# In a perfect world each browser should have 20912 results (10456 with 2 repetitions)
# Some have more due to repetitions/timeouts, ... (or because they were started more than once)
# Some have less as some tests did not run in at least one of the repetition 
# Currently we have 27 supported browsers (+ 3 failed ones: opera iOS, safari iOS, ucmobile android)
df.groupby(["automation_mode", "browser_id", "name", "version", "os", "headless_mode"])["id"].count().sort_values(ascending=False).to_frame().reset_index()

,automation_mode,browser_id,name,version,os,headless_mode,id
0,intent,27,brave,1.61 (23.12.18.22),iPadOS 17.2,real,35845
1,intent,31,operagx,2.2.2,iPadOS 17.2,real,29503
2,intent,29,firefox,121.0 (36782),iPadOS 17.2,real,26391
3,intent,32,edge,120.0.2210.116,iPadOS 17.2,real,25083
4,intent,26,chrome,120.0.6099.119,iPadOS 17.2,real,24939
5,intent,3,brave,1.60.116,Android 11,real,21594
6,selenium,35,safari,17.2,macOS 14.2.1,real,20960
7,selenium,39,firefox,120,macOS 14.2.1,headless,20916
8,selenium,15,edge,119,Ubuntu 22.04,headless-new,20912
9,selenium,14,firefox,119,Ubuntu 22.04,headless,20912


In [10]:
# Browsers with timeout results (test_status = 2)
# Results:
# - Many brave iPadOs
# - Some firefox_beta Android
# - Rare Ubuntu desktop
# - Some MacOS desktop (mostly Safari)
filtered_df = df.groupby(["automation_mode", "browser_id", "name", "version", "os", "headless_mode"])["test_status"].filter(lambda x: x.nunique() >= 2)
display(df[["automation_mode", "browser_id", "name", "version", "os", "headless_mode", "test_status"]].loc[df.index.isin(filtered_df.index)].groupby(["automation_mode", "browser_id", "name", "version", "os", "headless_mode"])["test_status"].value_counts().to_frame())

count
automation_mode browser_id name         version                   os           headless_mode test_status       
intent          4          ucmobile     13.3.8.1305               Android 11   real          0              206
                                                                                             2               47
                23         firefox_beta 122.0b1                   Android 11   real          0            19973
                                                                                             2               86
                27         brave        1.61 (23.12.18.22)        iPadOS 17.2  real          0            31001
                                                                                             2             4844
selenium        14         firefox      119                       Ubuntu 22.04 headless      0            20911
                                                                                             2                1
                15         edge         119                       Ubuntu 22.04 headless-new  0            20910
                                                                                             2                2
                19         firefox      119                       Ubuntu 22.04 xvfb          0            20907
                                                                                             2                5
                20         edge         119                       Ubuntu 22.04 xvfb          0            20902
                                                                                             2               10
                33         chrome       120                       macOS 14.2.1 real          0            20874
                                                                                             2                6
                34         firefox      120                       macOS 14.2.1 real          0            20911
                                                                                             2                1
                35         safari       17.2                      macOS 14.2.1 real          0            20683
                                                                                             2              277
                36         edge         120                       macOS 14.2.1 real          0            20907
                                                                                             2                5
                37         brave        1.60.118 (119.0.6045.163) macOS 14.2.1 real          0            20878
                                                                                             2                2
                38         chrome       120                       macOS 14.2.1 headless-new  0            20908
                                                                                             2                4
                39         firefox      120                       macOS 14.2.1 headless      0            20914
                                                                                             2                2
                40         edge         120                       macOS 14.2.1 headless-new  0            10542
                                                                                             2               18
                41         brave        1.60.118 (119.0.6045.163) macOS 14.2.1 headless-new  0            19306
                                                                                             2                6

In [11]:
# Tests that timed out!
# Mostly oac_iframe and referrer_iframe
# On MacOS also others: fullscreen_iframe, framing_*, imgloading_iframe, fetch_* (might have been a network issue or similar?)
display(df.loc[df["test_status"] == 2].groupby(["browser", "os", "test_name"])["id"].count().to_frame().sort_values(by="id", ascending=False))

id
browser                                            os           test_name                  
brave iPadOS 17.2 1.61 (23.12.18.22) intent real   iPadOS 17.2  oac_iframe             3108
                                                                referrer_iframe        1736
safari macOS 14.2.1 17.2 selenium real             macOS 14.2.1 referrer_iframe          83
firefox_beta Android 11 122.0b1 intent real        Android 11   oac_iframe               58
safari macOS 14.2.1 17.2 selenium real             macOS 14.2.1 oac_iframe               48
ucmobile Android 11 13.3.8.1305 intent real        Android 11   oac_iframe               45
safari macOS 14.2.1 17.2 selenium real             macOS 14.2.1 fullscreen_iframe        32
firefox_beta Android 11 122.0b1 intent real        Android 11   referrer_iframe          26
safari macOS 14.2.1 17.2 selenium real             macOS 14.2.1 framing_object           24
                                                                framing_embed            24
                                                                framing_iframe           24
                                                                oac_window.open          14
                                                                imgloading_iframe        12
                                                                fetch_GET                11
chrome macOS 14.2.1 120 selenium real              macOS 14.2.1 fetch_GET                 5
edge Ubuntu 22.04 119 selenium xvfb                Ubuntu 22.04 referrer_iframe           5
safari macOS 14.2.1 17.2 selenium real             macOS 14.2.1 fetch_TEST                5
firefox Ubuntu 22.04 119 selenium xvfb             Ubuntu 22.04 referrer_iframe           5
edge macOS 14.2.1 120 selenium real                macOS 14.2.1 fetch_GET                 5
edge macOS 14.2.1 120 selenium headless-new        macOS 14.2.1 framing_embed             5
                                                                framing_object            5
chrome macOS 14.2.1 120 selenium headless-new      macOS 14.2.1 fetch_GET                 4
edge macOS 14.2.1 120 selenium headless-new        macOS 14.2.1 framing_iframe            4
                                                                fetch_GET                 4
edge Ubuntu 22.04 119 selenium xvfb                Ubuntu 22.04 oac_window.open           3
ucmobile Android 11 13.3.8.1305 intent real        Android 11   oac_window.open           2
brave macOS 14.2.1 1.60.118 (119.0.6045.163) se... macOS 14.2.1 framing_object            2
                                                                framing_iframe            2
                                                                framing_embed             2
firefox_beta Android 11 122.0b1 intent real        Android 11   oac_window.open           2
edge Ubuntu 22.04 119 selenium xvfb                Ubuntu 22.04 oac_iframe                2
brave macOS 14.2.1 1.60.118 (119.0.6045.163) se... macOS 14.2.1 framing_embed             1
                                                                framing_object            1
firefox macOS 14.2.1 120 selenium headless         macOS 14.2.1 referrer_iframe           1
                                                                upgradeHSTS_subdomain     1
edge Ubuntu 22.04 119 selenium headless-new        Ubuntu 22.04 oac_window.open           1
chrome macOS 14.2.1 120 selenium real              macOS 14.2.1 fetch_TEST                1
edge Ubuntu 22.04 119 selenium headless-new        Ubuntu 22.04 referrer_iframe           1
firefox Ubuntu 22.04 119 selenium headless         Ubuntu 22.04 referrer_iframe           1
firefox macOS 14.2.1 120 selenium real             macOS 14.2.1 fetch_GET                 1

In [12]:
# Time taken!
# Time difference between first and last entry in the db for each browser
# "Broken" as we manually started the crawls twice with a break in between
df.groupby(["name", "version", "os", "headless_mode", "automation_mode", "org_scheme"])["created_at"].agg(["min", "max"]).apply(lambda x: x["max"] - x["min"], axis=1).to_frame()

0
name         version                   os           headless_mode automation_mode org_scheme                       
brave        1.59.120 (118.0.5993.88)  Ubuntu 22.04 headless-new  selenium        http       0 days 01:29:15.190542
                                                                                  https      0 days 01:26:48.371864
                                                    xvfb          selenium        http       0 days 01:30:28.246903
                                                                                  https      0 days 01:28:25.877146
             1.60.114 (119.0.6045.124) Ubuntu 22.04 headless-new  selenium        http       0 days 01:29:15.177088
                                                                                  https      0 days 01:26:53.931604
                                                    xvfb          selenium        http       0 days 01:30:25.268345
                                                                                  https      0 days 01:28:07.876485
             1.60.116                  Android 11   real          intent          http       0 days 02:18:48.616714
                                                                                  https      0 days 02:18:40.689261
             1.60.118 (119.0.6045.163) macOS 14.2.1 headless-new  selenium        http       0 days 02:08:29.862199
                                                                                  https      0 days 02:05:36.909407
                                                    real          selenium        http       0 days 19:35:19.750973
                                                                                  https      0 days 02:16:41.545086
             1.61 (23.12.18.22)        iPadOS 17.2  real          intent          http       0 days 19:28:40.622572
                                                                                  https      0 days 18:00:33.969640
chrome       119                       Ubuntu 22.04 headless-new  selenium        http       0 days 01:29:06.393217
                                                                                  https      0 days 01:26:43.887262
                                                    xvfb          selenium        http       0 days 01:30:11.077591
                                                                                  https      0 days 01:28:03.952745
             120                       macOS 14.2.1 headless-new  selenium        http       0 days 02:19:25.597890
                                                                                  https      0 days 02:16:03.257209
                                                    real          selenium        http       0 days 02:20:02.968239
                                                                                  https      0 days 02:16:29.342292
             120.0.6099.119            iPadOS 17.2  real          intent          http       0 days 23:02:28.097676
                                                                                  https      0 days 01:26:58.776521
             120.0.6099.44             Android 11   real          intent          http       0 days 05:28:40.875672
                                                                                  https      0 days 05:28:32.296819
edge         119                       Ubuntu 22.04 headless-new  selenium        http       0 days 01:23:53.035976
                                                                                  https      0 days 01:27:01.534731
                                                    xvfb          selenium        http       0 days 01:24:54.645264
                                                                                  https      0 days 01:28:17.297944
             120                       macOS 14.2.1 headless-new  selenium        http       0 days 17:33:37.276600
                                                      

## Test Metadata

- We have 10456 unique tests: (test_name x relation_info) x org_scheme x (resp_scheme x resp_host) x response_id)
- Each should have run 2x in each browser

In [13]:
# Unique tests
tests = {
    "Fetch": [4, 2, 8, 6], 
    "Framing": [9, 2, 8, 35], # 7+11+17
    "PP": [4, 2, 8, 24],
    "OAC": [3, 2, 8, 14],
    "TAO": [1, 2, 8, 13],
    "RP": [2, 2, 8, 16],
    "CSP-SCRIPT": [2, 2, 8, 17],
    "COEP": [2, 2, 8, 11],
    "CORP": [4, 2, 8, 11],
    "CSP-IMG": [1, 2, 8, 17],
    "HSTS": [2, 1, 2, 14],
    "COOP": [1, 2, 8, 11],
}
tests = pd.DataFrame.from_dict(tests, columns=["tests", "org_origins", "resp_origins", "responses"], orient="index")
tests["All tests"] = tests.prod(axis=1)
tests.loc["Sum"] = tests.sum(axis=0)
tests

,tests,org_origins,resp_origins,responses,All tests
Fetch,4,2,8,6,384
Framing,9,2,8,35,5040
PP,4,2,8,24,1536
OAC,3,2,8,14,672
TAO,1,2,8,13,208
RP,2,2,8,16,512
CSP-SCRIPT,2,2,8,17,544
COEP,2,2,8,11,352
CORP,4,2,8,11,704
CSP-IMG,1,2,8,17,272


## Result analysis
- Only consider the tests that did not timeout!
- (Use create_repeat.py and run them until each test has at least one result that is not timeout for each test!)

In [14]:
df_org = df
df = df.loc[df["test_status"] == 0]

### Stability analysis
- Does the same browser/env always result in the same outcomes?
- We ran all tests on linux, mac, android, iOS twice, does any of these tests have more than one result?

- Android:
    - 71/10456 have more than one outcome
    - Firefox beta: 47x subresourceloadingCOEP_img (same issue as on Linux?; swag-same-origin.jpg: error vs load; also swag-same-site.jpg: error vs load)
    - Brave: 10x referrer_iframe (empty vs http://headers.websec.saarland/; http vs https), 10x oac_window.open (oac true vs false), 2x accesswindow_direct (window.open.opener null vs object Window)
    - Chrome: 2x referrer_iframe (http vs https)
- Linux:
    - 30/10456 tests have different outcomes
    - Firefox: 30x subresourceloadingCOEP_img, all differences: 'swag-same-origin.jpg': 'error' | 'load'
        - trying to manually reproduce: e.g., http://sub.headers.websec.saarland/_hp/tests/subresource-loading-coep.sub.html?resp_type=basic&browser_id=1&label=COEP&first_id=57&last_id=57&scheme=http
        - devtools open: with cache disabeled: 15x error, without cache disabled: 13x or 16x error?
        - devtools closed: 13x or 16x error (tried to fix with Cache-Control: no-cache, no-store but does not seem to have an effect)
        - chrome always 14x error
- iOS/iPadOS:
    - 962/10456 have more than one outcome
    - Firefox: 792x fullscreen_iframe (TODO: investigate, number of tests here is 1536); (fullscreenEnabled: true vs fullscreenEnabled: undefined)
    - Operagx: 162x referrer_iframe (TODO: investigate, number of tests here is 512?) (empty vs http://headers.webappsec.eu/ vs fullURL)
    - Brave + Edge: subresourceloadingCOEP_img + oac_window.open + referrer_iframe for a total of 8 (message timeout vs other results? and swag.jpg load | error)
- MacOS:
    - 252/10456 have more than one outcome
    - framing_*: firefox and edge (TODO: investigate?) (message send vs message timeout)
    - subresourceloadingCOEP_img, subresourceloadingCORP_img, ... (Firefox same as on Linux?! + message timeout in edge/chrome/brave)
    - some others
        - accesswindow_direct: chrome, edge, safari window.open.opener null vs window
        - ...

In [15]:
for _, os_group in df.groupby("os"):
    stab = os_group.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "browser"])[["outcome_str"]].agg(["count", "nunique"])
    # Tests with more than one outcome
    diff_outcomes = stab.loc[stab[("outcome_str", "nunique")] != 1]
    print(f"{len(diff_outcomes)} tests have different outcomes!")
    display(diff_outcomes.reset_index()["browser"].value_counts().to_frame())
    display(diff_outcomes.reset_index()[["test_name", "browser"]].value_counts().to_frame())

    diff_with_responses = diff_outcomes.reset_index().groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "browser"])["response_id"].agg(list).to_frame().reset_index()


    for id, (test_name, org_scheme, org_host, resp_scheme, resp_host, relation_info, browser, response_ids) in diff_with_responses.iterrows():
        # TODO: shortcut, only show th first result for each test_name/browser; results are usually always the same!
        print(test_name, response_ids)
        rows = df.loc[(df["org_scheme"] == org_scheme) & (df["test_name"] == test_name) & (df["resp_scheme"] == resp_scheme) & (df["resp_host"] == resp_host) & (df["relation_info"] == relation_info) & (df["response_id"].isin(response_ids)) & (df["browser"] == browser)]
        with pd.option_context("display.max_colwidth", 200):
            display(rows[["outcome_str", "name"]].value_counts().to_frame())
            disp = rows.drop_duplicates(subset="outcome_str")[["browser", "outcome_str", "raw_header", "test_name", "org_scheme", "org_host", "resp_scheme", "relation_info", "resp_host", "full_url"]]
            disp = disp.style.format({'full_url': make_clickable})
            display(disp)
        # input("Continue!")
        


70 tests have different outcomes!


,count
browser,
firefox_beta Android 11 122.0b1 intent real,47
brave Android 11 1.60.116 intent real,22
chrome Android 11 120.0.6099.44 intent real,1


,,count
"(test_name, )","(browser, )",
subresourceloadingCOEP_img,firefox_beta Android 11 122.0b1 intent real,47
referrer_iframe,brave Android 11 1.60.116 intent real,10
oac_window.open,brave Android 11 1.60.116 intent real,10
accesswindow_direct,brave Android 11 1.60.116 intent real,2
referrer_iframe,chrome Android 11 120.0.6099.44 intent real,1


accesswindow_direct [72, 80]


,,count
outcome_str,name,
{'window.open.opener': 'null'},brave,3
"{'window.open.opener': 'object ""[object Window]""'}",brave,2


...

,,count
outcome_str,name,
error,brave,1
message timeout,brave,1


,browser,outcome_str,raw_header,test_name,org_scheme,org_host,resp_scheme,relation_info,resp_host,full_url
505871,brave macOS 14.2.1 1.60.118 (119.0.6045.163) selenium headless-new,message timeout,"[['Cross-Origin-Resource-Policy', 'same-site'], ['location', 'https://sub.headers.websec.saarland/_hp/common/empty.html']]",subresourceloadingCORP_img,https,sub.headers.websec.saarland,https,nested,sub.sub.headers.websec.saarland,https://sub.headers.websec.saarland/_hp/tests/subresource-loading-corp.sub.html?timeout=5&resp_type=basic&browser_id=1&label=CORP&first_id=55&last_id=55&scheme=https
582563,brave macOS 14.2.1 1.60.118 (119.0.6045.163) selenium headless-new,error,"[['Cross-Origin-Resource-Policy', 'same-site'], ['location', 'https://sub.headers.websec.saarland/_hp/common/empty.html']]",subresourceloadingCORP_img,https,sub.headers.websec.saarland,https,nested,sub.sub.headers.websec.saarland,https://sub.headers.websec.saarland/_hp/tests/subresource-loading-corp.sub.html?timeout=5&resp_type=basic&browser_id=1&label=CORP&first_id=55&last_id=55&scheme=https


subresourceloadingCORP_img [55]


,,count
outcome_str,name,
error,chrome,1
message timeout,chrome,1


,browser,outcome_str,raw_header,test_name,org_scheme,org_host,resp_scheme,relation_info,resp_host,full_url
505851,chrome macOS 14.2.1 120 selenium headless-new,message timeout,"[['Cross-Origin-Resource-Policy', 'same-site'], ['location', 'https://sub.headers.websec.saarland/_hp/common/empty.html']]",subresourceloadingCORP_img,https,sub.headers.websec.saarland,https,nested,sub.sub.headers.websec.saarland,https://sub.headers.websec.saarland/_hp/tests/subresource-loading-corp.sub.html?timeout=5&resp_type=basic&browser_id=1&label=CORP&first_id=55&last_id=55&scheme=https
608437,chrome macOS 14.2.1 120 selenium headless-new,error,"[['Cross-Origin-Resource-Policy', 'same-site'], ['location', 'https://sub.headers.websec.saarland/_hp/common/empty.html']]",subresourceloadingCORP_img,https,sub.headers.websec.saarland,https,nested,sub.sub.headers.websec.saarland,https://sub.headers.websec.saarland/_hp/tests/subresource-loading-corp.sub.html?timeout=5&resp_type=basic&browser_id=1&label=CORP&first_id=55&last_id=55&scheme=https


### Differences between browsers analysis:
- Only one row for each browser and remove browsers that currently have almost no results
- Most tests have only one outcome (7518/10456)
- Some have more: 1836 have 2, ...


In [16]:
df_stab = df

# Only keep one row for each test
# If the test had non-deterministic results, chose the first?
print("Original data entries", len(df))
df = df.drop_duplicates(subset=["test_name", "relation_info", "browser", "org_origin", "resp_origin", "response_id"])
print("Only one row for each test (per browser)", len(df))

# Remove browsers that currently do not have almost all results (safari + opera iOS, ucmobile android)
df = df.loc[~df["browser_id"].isin([4, 28, 30])]
print("Remove failed browsers", len(df))

Original data entries 605832
Only one row for each test (per browser) 292461
Remove failed browsers 292178


In [17]:
res = df.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"])
res = res["outcome_str"].agg("nunique")
display(res.value_counts().to_frame())

,count
outcome_str,
1,7518
2,1836
3,863
4,236
5,3


- fetch_GET, fetch_TEST, upgradeHSTS_subdomain, script_execution_iframe_direct, subresourceloadingCORP_img_sandbox, framing_iframe always have the same result
- most others have different results from rarely to common
- subresourceloadingCORP_object, accesswindow_direct always have different results

In [18]:
r = res.reset_index()
r["diff"] = r["outcome_str"] != 1
r.groupby(["test_name", "relation_info"])["diff"].agg(["count", "mean"]).sort_values("mean")

count      mean
test_name                     relation_info                  
fetch_GET                     credentials        96  0.000000
                              custom_headers     96  0.000000
                              simple             96  0.000000
fetch_TEST                    custom_method      96  0.000000
framing_iframe                direct            560  0.000000
subresourceloadingCORP_img    sandbox           176  0.000000
script_execution_iframe       direct            272  0.000000
upgradeHSTS_subdomain         subdomain          28  0.000000
framing_iframe                nested            560  0.007143
                              sandbox           560  0.014286
subresourceloadingCORP_img    nested            176  0.022727
                              direct            176  0.028409
upgradeHSTS_direct            direct             28  0.035714
subresourceloadingCOEP_img    sandbox           176  0.045455
imgloading_iframe             direct            272  0.062500
referrer_iframe               iframe            256  0.070312
script_execution_iframe       sandbox           272  0.077206
framing_embed                 nested            560  0.214286
framing_object                nested            560  0.214286
framing_embed                 sandbox           560  0.214286
framing_object                sandbox           560  0.214286
perfAPI_img                   direct            208  0.264423
framing_embed                 direct            560  0.332143
framing_object                direct            560  0.332143
fullscreen_iframe             child_sandbox     384  0.458333
                              child_allow       384  0.458333
                              child             384  0.458333
subresourceloadingCOEP_img    direct            176  0.551136
oac_iframe                    direct            224  0.642857
                              sandbox           224  0.642857
fullscreen_iframe             direct            384  0.687500
oac_window.open               window.open       224  0.857143
referrer_iframe               window.open       256  0.875000
accesswindow_direct           direct            176  1.000000
subresourceloadingCORP_object direct            176  1.000000

In [19]:
# Select only test rows with more than one outcome
tests_with_mult_outcomes = r.loc[r["diff"]].set_index(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"])
df_mult = df.loc[df[["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"]].apply(tuple, axis=1).isin(tests_with_mult_outcomes.index)]

print(f"All test rows: {len(df)}, Tests with more than one outcome: {len(df_mult)}")


All test rows: 292178, Tests with more than one outcome: 82164


In [20]:
# Display difference groups
for grouping, group in df_mult.groupby(["test_name", "relation_info"]):
    print(grouping)
    df_list = []
    for _, one_test in group.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"]):
        browsers = one_test.groupby("outcome_str")["browser"].unique().apply(list).apply(sorted).to_frame()
        new = True
        for df_b in df_list:
            if df_b.equals(browsers):
                new = False
        if new:
            df_list.append(browsers)
    print(len(df_list))
    for df_b in df_list:
        with pd.option_context("display.max_colwidth", 1000):
            display(df_b)
    #input("Continue!")

('accesswindow_direct', 'direct')
5


,browser
outcome_str,
"{'window.open.opener': 'object ""TypeError: null is not an object (evaluating \'w.opener\')""'}",[brave iPadOS 17.2 1.61 (23.12.18.22) intent real]
"{'window.open.opener': 'object ""[object Window]""'}","[brave Android 11 1.60.116 intent real, brave Ubuntu 22.04 1.59.120 (118.0.5993.88) selenium headless-new, brave Ubuntu 22.04 1.59.120 (118.0.5993.88) selenium xvfb, brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium headless-new, brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium xvfb, brave macOS 14.2.1 1.60.118 (119.0.6045.163) selenium headless-new, brave macOS 14.2.1 1.60.118 (119.0.6045.163) selenium real, chrome Android 11 120.0.6099.44 intent real, chrome Ubuntu 22.04 119 selenium headless-new, chrome Ubuntu 22.04 119 selenium xvfb, chrome iPadOS 17.2 120.0.6099.119 intent real, chrome macOS 14.2.1 120 selenium headless-new, chrome macOS 14.2.1 120 selenium real, edge Ubuntu 22.04 119 selenium headless-new, edge Ubuntu 22.04 119 selenium xvfb, edge iPadOS 17.2 120.0.2210.116 intent real, edge macOS 14.2.1 120 selenium headless-new, edge macOS 14.2.1 120 selenium real, firefox Ubuntu 22.04 119 selenium headless, firefox Ubuntu 22.04 119 selenium xvfb, firefox iPadOS 17..."


,browser
outcome_str,
{'window.open.opener': 'null'},"[brave Android 11 1.60.116 intent real, brave Ubuntu 22.04 1.59.120 (118.0.5993.88) selenium headless-new, brave Ubuntu 22.04 1.59.120 (118.0.5993.88) selenium xvfb, brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium headless-new, brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium xvfb, brave macOS 14.2.1 1.60.118 (119.0.6045.163) selenium headless-new, brave macOS 14.2.1 1.60.118 (119.0.6045.163) selenium real, chrome Android 11 120.0.6099.44 intent real, chrome Ubuntu 22.04 119 selenium headless-new, chrome Ubuntu 22.04 119 selenium xvfb, chrome macOS 14.2.1 120 selenium headless-new, chrome macOS 14.2.1 120 selenium real, edge macOS 14.2.1 120 selenium headless-new, edge macOS 14.2.1 120 selenium real]"
"{'window.open.opener': 'object ""TypeError: null is not an object (evaluating \'w.opener\')""'}",[brave iPadOS 17.2 1.61 (23.12.18.22) intent real]
"{'window.open.opener': 'object ""[object Window]""'}","[chrome iPadOS 17.2 120.0.6099.119 intent real, edge Ubuntu 22.04 119 selenium headless-new, edge Ubuntu 22.04 119 selenium xvfb, edge iPadOS 17.2 120.0.2210.116 intent real, firefox Ubuntu 22.04 119 selenium headless, firefox Ubuntu 22.04 119 selenium xvfb, firefox iPadOS 17.2 121.0 (36782) intent real, firefox macOS 14.2.1 120 selenium headless, firefox macOS 14.2.1 120 selenium real, firefox_beta Android 11 122.0b1 intent real, opera Android 11 78.4.4143.75735 intent real, operagx iPadOS 17.2 2.2.2 intent real, safari macOS 14.2.1 17.2 selenium real]"


,browser
outcome_str,
{'window.open.opener': 'null'},"[brave Android 11 1.60.116 intent real, brave Ubuntu 22.04 1.59.120 (118.0.5993.88) selenium headless-new, brave Ubuntu 22.04 1.59.120 (118.0.5993.88) selenium xvfb, brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium headless-new, brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium xvfb, brave macOS 14.2.1 1.60.118 (119.0.6045.163) selenium headless-new, brave macOS 14.2.1 1.60.118 (119.0.6045.163) selenium real, chrome Android 11 120.0.6099.44 intent real, chrome Ubuntu 22.04 119 selenium headless-new, chrome Ubuntu 22.04 119 selenium xvfb, chrome macOS 14.2.1 120 selenium headless-new, chrome macOS 14.2.1 120 selenium real, edge macOS 14.2.1 120 selenium headless-new, edge macOS 14.2.1 120 selenium real]"
"{'window.open.opener': 'object ""TypeError: null is not an object (evaluating \'w.opener\')""'}",[brave iPadOS 17.2 1.61 (23.12.18.22) intent real]
"{'window.open.opener': 'object ""[object Window]""'}","[chrome iPadOS 17.2 120.0.6099.119 intent real, edge Ubuntu 22.04 119 selenium headless-new, edge Ubuntu 22.04 119 selenium xvfb, edge iPadOS 17.2 120.0.2210.116 intent real, firefox Ubuntu 22.04 119 selenium headless, firefox Ubuntu 22.04 119 selenium xvfb, firefox iPadOS 17.2 121.0 (36782) intent real, firefox macOS 14.2.1 120 selenium headless, firefox macOS 14.2.1 120 selenium real, opera Android 11 78.4.4143.75735 intent real, operagx iPadOS 17.2 2.2.2 intent real, safari macOS 14.2.1 17.2 selenium real]"


,browser
outcome_str,
{'window.open.opener': 'null'},"[brave Android 11 1.60.116 intent real, brave Ubuntu 22.04 1.59.120 (118.0.5993.88) selenium headless-new, brave Ubuntu 22.04 1.59.120 (118.0.5993.88) selenium xvfb, brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium headless-new, brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium xvfb, brave macOS 14.2.1 1.60.118 (119.0.6045.163) selenium headless-new, brave macOS 14.2.1 1.60.118 (119.0.6045.163) selenium real, chrome Android 11 120.0.6099.44 intent real, chrome Ubuntu 22.04 119 selenium headless-new, chrome Ubuntu 22.04 119 selenium xvfb, chrome iPadOS 17.2 120.0.6099.119 intent real, chrome macOS 14.2.1 120 selenium headless-new, chrome macOS 14.2.1 120 selenium real, edge Ubuntu 22.04 119 selenium headless-new, edge Ubuntu 22.04 119 selenium xvfb, edge iPadOS 17.2 120.0.2210.116 intent real, edge macOS 14.2.1 120 selenium headless-new, edge macOS 14.2.1 120 selenium real, firefox Ubuntu 22.04 119 selenium headless, firefox Ubuntu 22.04 119 selenium xvfb, firefox iPadOS 17..."
"{'window.open.opener': 'object ""TypeError: null is not an object (evaluating \'w.opener\')""'}",[brave iPadOS 17.2 1.61 (23.12.18.22) intent real]


...

,browser
outcome_str,
error,"[firefox Ubuntu 22.04 119 selenium headless, firefox Ubuntu 22.04 119 selenium xvfb, firefox macOS 14.2.1 120 selenium headless, firefox macOS 14.2.1 120 selenium real, firefox_beta Android 11 122.0b1 intent real]"
load,"[brave Android 11 1.60.116 intent real, brave Ubuntu 22.04 1.59.120 (118.0.5993.88) selenium headless-new, brave Ubuntu 22.04 1.59.120 (118.0.5993.88) selenium xvfb, brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium headless-new, brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium xvfb, brave macOS 14.2.1 1.60.118 (119.0.6045.163) selenium headless-new, brave macOS 14.2.1 1.60.118 (119.0.6045.163) selenium real, chrome Android 11 120.0.6099.44 intent real, chrome Ubuntu 22.04 119 selenium headless-new, chrome Ubuntu 22.04 119 selenium xvfb, chrome macOS 14.2.1 120 selenium headless-new, chrome macOS 14.2.1 120 selenium real, edge Ubuntu 22.04 119 selenium headless-new, edge Ubuntu 22.04 119 selenium xvfb, edge macOS 14.2.1 120 selenium headless-new, edge macOS 14.2.1 120 selenium real, opera Android 11 78.4.4143.75735 intent real]"
timeout: no event fired,"[brave iPadOS 17.2 1.61 (23.12.18.22) intent real, chrome iPadOS 17.2 120.0.6099.119 intent real, edge iPadOS 17.2 120.0.2210.116 intent real, firefox iPadOS 17.2 121.0 (36782) intent real, operagx iPadOS 17.2 2.2.2 intent real, safari macOS 14.2.1 17.2 selenium real]"


,browser
outcome_str,
load,"[brave Android 11 1.60.116 intent real, brave Ubuntu 22.04 1.59.120 (118.0.5993.88) selenium headless-new, brave Ubuntu 22.04 1.59.120 (118.0.5993.88) selenium xvfb, brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium headless-new, brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium xvfb, brave macOS 14.2.1 1.60.118 (119.0.6045.163) selenium headless-new, brave macOS 14.2.1 1.60.118 (119.0.6045.163) selenium real, chrome Android 11 120.0.6099.44 intent real, chrome Ubuntu 22.04 119 selenium headless-new, chrome Ubuntu 22.04 119 selenium xvfb, chrome macOS 14.2.1 120 selenium headless-new, chrome macOS 14.2.1 120 selenium real, edge Ubuntu 22.04 119 selenium headless-new, edge Ubuntu 22.04 119 selenium xvfb, edge macOS 14.2.1 120 selenium headless-new, edge macOS 14.2.1 120 selenium real, firefox Ubuntu 22.04 119 selenium headless, firefox Ubuntu 22.04 119 selenium xvfb, firefox macOS 14.2.1 120 selenium headless, firefox macOS 14.2.1 120 selenium real, opera Android 11 78.4.414..."
timeout: no event fired,"[brave iPadOS 17.2 1.61 (23.12.18.22) intent real, chrome iPadOS 17.2 120.0.6099.119 intent real, edge iPadOS 17.2 120.0.2210.116 intent real, firefox iPadOS 17.2 121.0 (36782) intent real, operagx iPadOS 17.2 2.2.2 intent real, safari macOS 14.2.1 17.2 selenium real]"


,browser
outcome_str,
error,"[brave Android 11 1.60.116 intent real, brave Ubuntu 22.04 1.59.120 (118.0.5993.88) selenium headless-new, brave Ubuntu 22.04 1.59.120 (118.0.5993.88) selenium xvfb, brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium headless-new, brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium xvfb, brave macOS 14.2.1 1.60.118 (119.0.6045.163) selenium headless-new, brave macOS 14.2.1 1.60.118 (119.0.6045.163) selenium real, chrome Android 11 120.0.6099.44 intent real, chrome Ubuntu 22.04 119 selenium headless-new, chrome Ubuntu 22.04 119 selenium xvfb, chrome macOS 14.2.1 120 selenium headless-new, chrome macOS 14.2.1 120 selenium real, edge Ubuntu 22.04 119 selenium headless-new, edge Ubuntu 22.04 119 selenium xvfb, edge macOS 14.2.1 120 selenium headless-new, edge macOS 14.2.1 120 selenium real, firefox Ubuntu 22.04 119 selenium headless, firefox Ubuntu 22.04 119 selenium xvfb, firefox macOS 14.2.1 120 selenium headless, firefox macOS 14.2.1 120 selenium real, firefox_beta Android 11 1..."
timeout: no event fired,"[brave iPadOS 17.2 1.61 (23.12.18.22) intent real, chrome iPadOS 17.2 120.0.6099.119 intent real, edge iPadOS 17.2 120.0.2210.116 intent real, firefox iPadOS 17.2 121.0 (36782) intent real, operagx iPadOS 17.2 2.2.2 intent real, safari macOS 14.2.1 17.2 selenium real]"


('upgradeHSTS_direct', 'direct')
1


,browser
outcome_str,
{'response.redirected': False},"[brave Android 11 1.60.116 intent real, brave Ubuntu 22.04 1.59.120 (118.0.5993.88) selenium headless-new, brave Ubuntu 22.04 1.59.120 (118.0.5993.88) selenium xvfb, brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium headless-new, brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium xvfb, brave macOS 14.2.1 1.60.118 (119.0.6045.163) selenium headless-new, brave macOS 14.2.1 1.60.118 (119.0.6045.163) selenium real, chrome Android 11 120.0.6099.44 intent real, chrome Ubuntu 22.04 119 selenium headless-new, chrome Ubuntu 22.04 119 selenium xvfb, chrome macOS 14.2.1 120 selenium headless-new, chrome macOS 14.2.1 120 selenium real, edge Ubuntu 22.04 119 selenium headless-new, edge Ubuntu 22.04 119 selenium xvfb, edge macOS 14.2.1 120 selenium headless-new, edge macOS 14.2.1 120 selenium real, firefox Ubuntu 22.04 119 selenium headless, firefox Ubuntu 22.04 119 selenium xvfb, firefox macOS 14.2.1 120 selenium headless, firefox macOS 14.2.1 120 selenium real, firefox_beta Android 11 1..."
{'response.redirected': True},"[brave iPadOS 17.2 1.61 (23.12.18.22) intent real, chrome iPadOS 17.2 120.0.6099.119 intent real, edge iPadOS 17.2 120.0.2210.116 intent real, firefox iPadOS 17.2 121.0 (36782) intent real, operagx iPadOS 17.2 2.2.2 intent real, safari macOS 14.2.1 17.2 selenium real]"


In [46]:
with pd.option_context("display.max_colwidth", None):
    #display(tree_df.loc[(tree_df["test_name"] == "upgradeHSTS_direct") & (tree_df["relation_info"] == "direct")][["full_url", "browser"]].style.format({"full_url": make_clickable}))
    pass



,full_url,browser
48134,http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?resp_type=basic&browser_id=1&label=HSTS&first_id=140&last_id=140&scheme=http,edge Ubuntu 22.04 119 selenium headless-new
51576,http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?resp_type=basic&browser_id=1&label=HSTS&first_id=140&last_id=140&scheme=http,edge Ubuntu 22.04 119 selenium xvfb
51642,http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?resp_type=basic&browser_id=1&label=HSTS&first_id=140&last_id=140&scheme=http,firefox Ubuntu 22.04 119 selenium headless
55968,http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?resp_type=basic&browser_id=1&label=HSTS&first_id=140&last_id=140&scheme=http,firefox Ubuntu 22.04 119 selenium xvfb
65179,http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?resp_type=basic&browser_id=1&label=HSTS&first_id=140&last_id=140&scheme=http,chrome Ubuntu 22.04 119 selenium headless-new
65452,http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?resp_type=basic&browser_id=1&label=HSTS&first_id=140&last_id=140&scheme=http,brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium headless-new
65989,http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?resp_type=basic&browser_id=1&label=HSTS&first_id=140&last_id=140&scheme=http,brave Ubuntu 22.04 1.59.120 (118.0.5993.88) selenium headless-new
70127,http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?resp_type=basic&browser_id=1&label=HSTS&first_id=140&last_id=140&scheme=http,chrome Ubuntu 22.04 119 selenium xvfb
71048,http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?resp_type=basic&browser_id=1&label=HSTS&first_id=140&last_id=140&scheme=http,brave Ubuntu 22.04 1.59.120 (118.0.5993.88) selenium xvfb
71106,http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?resp_type=basic&browser_id=1&label=HSTS&first_id=140&last_id=140&scheme=http,brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium xvfb


In [513]:
# Other approach to list the difference groups
def get_uniques(df):
    browsers = f"{df['name'].unique().tolist()}"
    try:
        outcome = unique_outcomes[df.name[0]]
    except KeyError:
        outcome = {}
        #outcome = []
    outcome[browsers] = df.name[1]
    #outcome.append(f"{df.name[1]} ({browsers})")
    unique_outcomes[df.name[0]] = outcome

for grouping, group in df_mult.groupby(["test_name", "relation_info"]):
    
    unique_outcomes = {}
    group.groupby(["name"])["outcome_str"].value_counts(normalize=False).reset_index().groupby(["outcome_str", "count"], group_keys=True).apply(get_uniques)
    d = pd.DataFrame.from_dict(unique_outcomes, orient="index")
    if d.shape[1] != 1:
        display(d)
        print()

,"['firefox_beta', 'opera', 'operagx', 'safari']",['firefox'],['edge'],['chrome'],['brave'],['firefox_beta'],"['opera', 'operagx', 'safari']"
{'window.open.opener': 'null'},32.0,160.0,216.0,332.0,416,NaN,NaN
"{'window.open.opener': 'object ""[object Window]""'}",NaN,720.0,664.0,724.0,816,143.0,144.0
"{'window.open.opener': 'object ""TypeError: null is not an object (evaluating \'w.opener\')""'}",NaN,NaN,NaN,NaN,176,NaN,NaN


,['firefox_beta'],['opera'],"['edge', 'firefox']",['chrome'],['brave'],"['brave', 'chrome', 'edge', 'firefox', 'operagx', 'safari']"
message send,185.0,186.0,744.0,930.0,1302.0,NaN
message timeout,NaN,NaN,NaN,NaN,NaN,186.0


,['firefox_beta'],['opera'],['edge'],['firefox'],['chrome'],['brave'],"['brave', 'chrome', 'firefox', 'operagx', 'safari']"
message send,119.0,120.0,476,480.0,600.0,840.0,NaN
message timeout,NaN,NaN,124,NaN,NaN,NaN,120.0


,['firefox_beta'],['opera'],['edge'],['firefox'],['chrome'],['brave'],"['brave', 'chrome', 'firefox', 'operagx', 'safari']"
message send,119.0,120.0,476,480.0,600.0,840.0,NaN
message timeout,NaN,NaN,124,NaN,NaN,NaN,120.0


,"['firefox_beta', 'opera', 'operagx', 'safari']",['edge'],['firefox'],['chrome'],['brave']
message send,4.0,16,20.0,24.0,32.0
message timeout,NaN,4,NaN,NaN,NaN


...

,['brave'],"['chrome', 'operagx', 'safari']","['operagx', 'safari']","['firefox_beta', 'opera']","['edge', 'firefox']",['chrome']
message timeout,4,NaN,NaN,NaN,NaN,NaN
"{'image-events': {'swag.jpg': 'error', 'swag-same-site.jpg': 'error', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}",1,4.0,NaN,NaN,NaN,NaN
"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'error', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}",59,NaN,4.0,8.0,40.0,44.0


,['opera'],"['firefox_beta', 'operagx', 'safari']",['edge'],['chrome'],['firefox'],['brave']
error,4.0,5.0,20.0,24.0,25.0,29
load,1.0,NaN,5.0,6.0,NaN,7
timeout: no event fired,NaN,NaN,NaN,NaN,NaN,4


,"['firefox_beta', 'opera', 'operagx', 'safari']","['edge', 'firefox']",['chrome'],['brave']
error,4.0,20.0,22,28
message timeout,NaN,NaN,2,4


,['opera'],['firefox_beta'],['edge'],['chrome'],['firefox'],['brave'],"['brave', 'chrome', 'edge', 'firefox', 'operagx', 'safari']"
error,44.0,76.0,176.0,220.0,304.0,308.0,NaN
load,132.0,92.0,528.0,660.0,400.0,924.0,NaN
timeout: no event fired,NaN,NaN,NaN,NaN,NaN,NaN,176.0


,"['firefox_beta', 'opera']","['edge', 'firefox']",['chrome'],['brave'],"['brave', 'chrome', 'edge', 'firefox', 'operagx', 'safari']"
{'response.redirected': False},1.0,4.0,5.0,7.0,NaN
{'response.redirected': True},NaN,NaN,NaN,NaN,1.0


#### Differences
- Tree analysis (2024-01-10)
- Difference groups
- accesswindow_direct (COOP)
    - [ ] Brave iPadOS: {'window.open.opener': 'object "TypeError: null is not an object (evaluating \'w.opener\')"'}; sometimes?
    - [ ] Brave (Ubuntu,  macOS) ,  Chrome  (Android, Ubuntu, macOS), Edge (macOS or version 120): COOP first?
    - [ ] Brave (Android), Chrome (IPadOS), Edge (Ubuntu or version 119, IPadOS), Firefox, Safari, Opera: redirect first?
    - [ ] Maybe also another issue in addition? `tree_df.loc[(tree_df["test_name"] == "accesswindow_direct")].groupby("status_code")[["outcome_str", "os"]].value_counts().to_frame()`
- framing_embed_direct, framing_object_direct
    - [x] (PostMessages on) Embed and Object not supported on Safari and all iPadOS browsers (webkit)
- framing_object_sandbox/nested, framing_embed_sandbox/nested
    - [x] same as above + probably only a timeout issue with our tests on edge macOS (8/426 tests)
- framing_iframe_nested
    - [x] probably only a timeout issue with our tests on edge macOS (4x)
- framing_iframe_sandbox
    - [x] Firefox (Android, Ubuntu, macOS): CSP-FA *; allowed (message send), other browsers deny
    - [ ] TODO: report bug on bugzilla?!
- fullscreen_iframe_child (PP)
    - [ ] sometimes undefined in firefox (iPadOS) and operagx (iPadOS)
    - [ ] PP not supported in Firefox (macOS, Ubuntu, iPadOS, android), safari + iPadOS browsers?
    - [ ] Brave iPadOS issue? (fullscreen allowed for wrong origin?)
    - [ ] FP scheme not checked corretly firefox dekstop + iPadOS? (not sure)
- fullscreen_iframe_child_allow (PP)
    - [ ] similar to above?
- fullscreen_iframe_child_sandbox (PP)
    - [ ] similar to above? undefined firefox + operagx (both iPadOS), allowed only for brave iPadOS?
- fullscreen_iframe_direct (PP)
    - [ ] similar to above, not supported Firefox, Webkit; undefined for some reason in firefox + operagx (both iPadOS)
- imgloading_iframe_direct (CSP)
    - [ ] probably one timeout error in our tests firefox macOS
    - [ ] Chromium-based browsers autoupgrade http to https! (both response origin and orginal origin?)
    - [ ] img-src *.headers.websec.saarland interpreted differently?
- oac_iframe_direct (OAC)
    - [x] OAC not supported in Webkit + Firefox based browsers
    - [ ] Brave Ubuntu + Opera (Android) other results? (maybe because they use older chromium versions: 118 and 117)
    - [ ] ...
- oac_iframe_sandbox (OAC)
    - [x] OAC not supported in WebKit + Firefox based browsers
- oac_window.open_window.open (OAC)
    - [x] OAC not supported
    - [ ] Brave iPadOS timeout issue
    - [ ] Edge Ubuntu, Opera, Brave Ubuntu sometimes false while others are true?
- perfAPI_img_direct (TAO)
    - [ ] Firefox, Safari, Brave (iPadOS), OperaGX (iPadOS) behave differently?
    - [ ] Intrestingly edge + chrome (iPadOS) diverge from other Webkit browsers?
- referrer_iframe_iframe (RP)
    - [x] Brave Ubuntu + macOS never leaks the full_url!
    - [ ] In some cases brave (Android), Chrome (Android, Ubuntu, macOS), edge (Ubuntu, MacOS), opera (android) leak the full_url while the other browsers do not!
        - Reason 1 seems to be unsafe-url only supported there
        - Reason 2 seems to be no-referrer-when-downgrade only supported there
        - In a list the last supported one is used?, so they use unsafe-url
        - [ ] TODO: report the issue for brave Android? the other browsers act as expected? **Might have also be an issue during debugging/wrong browser id?; probably best to rerun all android browsers again?!**
        - `tree_df.loc[(tree_df["test_name"] == "referrer_iframe") & (tree_df["relation_info"] == "iframe") & (tree_df["resp_origin"] == "https://headers.webappsec.eu")].groupby(["raw_header", "outcome_str"])["browser"].unique().apply(sorted).to_frame()`
- referrer_iframe_window.open (RP)
    - Complicated stuff; some timeouts on iPadOS; some differences due to HTTPS upgrades?!
    - [ ] no referrer at all on brave (and operagx) sometimes?
    - [ ] ...
- script_execution_iframe_sandbox (CSP)
    - [ ] Safari sandbox bug? (report) (maybe related https://bugs.webkit.org/show_bug.cgi?id=223848 unsure though)
    - [ ] All iPadOS and Safari MacOS execute for `script-src 'self'` but not for `script-src *.headers.websec.saarland`
- subresourceloadingCOEP_img_direct (COEP)
    - [ ] Firefox "random" behavior (see stability analysis; sometimes swag-same-origin.jpg loads even though it should not, sometimes swag-same-site.jpg loads (or errors) though it should not
    - [ ] All WebKit-based browsers also sometimes error instead of load?
    - [ ] (Also some timeouts brave macOS)
- subresourceloadingCOEP_img_sandbox (COEP)
    - [ ] Something special with Webkit-based browsers (error swag.jpg even though it should load?)
- subresourceloadingCORP_img_direct (CORP)
    - [ ] Safari, most WebKit-based browsers (iPadOS) and Firefox (all systems) error for CORP same-origin with resp_orgin http://sub.headers.websec.saarland (Probably due to HTTP auto upgrades?!)
    - `display(tree_df.loc[(tree_df["test_name"] == "subresourceloadingCORP_img") & (tree_df["relation_info"] == "direct")].groupby(["raw_header", "resp_origin", "org_origin", "outcome_str"])["browser"].unique().apply(sorted).to_frame())`
- subresourceloadingCORP_img_nested (CORP)
    - [ ] Timeout issue on macOS; no real difference?
- subresourceloadingCORP_object_direct (CORP)
    - [ ] Firefox applies CORP on object tags (even though it should not) (TODO: report, again, I already saw this for the XS-Leaks paper?)
    - [ ] Safari/WebKit no support for object/pM on object
- upgradeHSTS_direct_direct (HSTS)
    - Safari/Webkit: `[['Strict-Transport-Security', 'max-age=20, max-age=20; includeSubDomains, includeSubDomains, , max-age=20; includeSubDomains; preload, max-age=0, max-age=-5']]` redirects even though it should not?
    - Sending `[["Strict-Transport-Security", "max-age=20"], ["Strict-Transport-Security", "max-age=20; includeSubDomains"], ["Strict-Transport-Security", "includeSubDomains"], ["Strict-Transport-Security", ""], ["Strict-Transport-Security", "max-age=20; includeSubDomains; preload"], ["Strict-Transport-Security", "max-age=0"], ["Strict-Transport-Security", "max-age=-5"]]` redirect in all browsers. The spec says take the first. Safari probably first splits and then takes the first, the other browsers do not split and the full string with Commas is not valid according to the syntax!
    - Safari: parse until the first comma
    - Firefox: always parse as invalid as comma not allowed within the header
    - Chromium-based: "max-age=60; includeSubDomains, max-age=60" -> max-age=60 + invalid directive; "max-age=60, includeSubDomains; max-age=60" -> invalid max-age value, parsing fails! (behavior still as described by siewert2022parsing; however Chrome acts against the specs as includesubdomains, is not an unknown token that should be ignored but an invalid one as token is not allowed to use comma!)
    - "includeSubDomains, max-age=60; max-age=60" -> only upgrades in Chromium-based browsers!
    - duplicate directives are not allowed in any browser (e.g., max-age=60; max-age=60)

In [150]:
tree_df.loc[(tree_df["test_name"] == "upgradeHSTS_direct")][["browser", "outcome_str", "full_url"]].style.format({"full_url": make_clickable})

,browser,outcome_str,full_url
48134,edge Ubuntu 22.04 119 selenium headless-new,{'response.redirected': False},http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?resp_type=basic&browser_id=1&label=HSTS&first_id=140&last_id=140&scheme=http
51576,edge Ubuntu 22.04 119 selenium xvfb,{'response.redirected': False},http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?resp_type=basic&browser_id=1&label=HSTS&first_id=140&last_id=140&scheme=http
51642,firefox Ubuntu 22.04 119 selenium headless,{'response.redirected': False},http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?resp_type=basic&browser_id=1&label=HSTS&first_id=140&last_id=140&scheme=http
55968,firefox Ubuntu 22.04 119 selenium xvfb,{'response.redirected': False},http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?resp_type=basic&browser_id=1&label=HSTS&first_id=140&last_id=140&scheme=http
65179,chrome Ubuntu 22.04 119 selenium headless-new,{'response.redirected': False},http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?resp_type=basic&browser_id=1&label=HSTS&first_id=140&last_id=140&scheme=http
65452,brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium headless-new,{'response.redirected': False},http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?resp_type=basic&browser_id=1&label=HSTS&first_id=140&last_id=140&scheme=http
65989,brave Ubuntu 22.04 1.59.120 (118.0.5993.88) selenium headless-new,{'response.redirected': False},http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?resp_type=basic&browser_id=1&label=HSTS&first_id=140&last_id=140&scheme=http
70127,chrome Ubuntu 22.04 119 selenium xvfb,{'response.redirected': False},http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?resp_type=basic&browser_id=1&label=HSTS&first_id=140&last_id=140&scheme=http
71048,brave Ubuntu 22.04 1.59.120 (118.0.5993.88) selenium xvfb,{'response.redirected': False},http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?resp_type=basic&browser_id=1&label=HSTS&first_id=140&last_id=140&scheme=http
71106,brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium xvfb,{'response.redirected': False},http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?resp_type=basic&browser_id=1&label=HSTS&first_id=140&last_id=140&scheme=http


### Similarity of browsers
- Percentage and absolute differences between the browsers
- If two browsers/settings are exactly the same, we do not have to test all of them anymore
- Due to some timeouts and other errors almost no browsers are exactly the same? maybe define some error threshold?
- Exactly the same:
    - brave ubuntu 118 headless vs xvfb
    - brave ubuntu 119 headless vs xvfb
    - chrome ubuntu 119 headles vs xvfb

In [148]:
# Function to calculate absolute and percentage overlap
def calculate_overlap(df, col1, col2):
    df = df[[col1, col2]].dropna()
    df.columns.values[0] = 0
    df.columns.values[1] = 1
    absolute_overlap = (df[0] == df[1]).sum()
    unique_rows = len(df)
    diff_rows = unique_rows - absolute_overlap
    percentage_diff = (diff_rows / unique_rows) * 100
    return diff_rows, percentage_diff, unique_rows


def display_overlap(df, show_all=True, name="All"):
    print(name)
    sim_frame = df.set_index(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "browser"])["outcome_str"].unstack().astype("str")
    sim_frame = sim_frame.reset_index().drop(columns=["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"])
    # Create a dictionary to store results
    results = {}
    matrix = {}
    
    # Iterate through all pairs of columns
    for i, col1 in enumerate(sim_frame.columns):
        for col2 in sim_frame.columns:
            key = f"{col1}_vs_{col2}"
            results[key] = calculate_overlap(sim_frame, col1, col2)
            try:
                l = matrix[col1]
            except KeyError:
                l = {}
            # 0 for absolute count of differences, 1 for percentage
            l[col2] = results[key][0]
            matrix[col1] = l
            
    
    # Convert the results to a DataFrame for better visualization
    results_df = pd.DataFrame.from_dict(results, orient='index', columns=['Absolute Diff', 'Percentage Diff', "Count"])
    
    # Display the results
    if show_all:
        display(results_df)
    
    # Display difference matrix
    with pd.option_context("display.max_columns", 28):
        if show_all:
            display(pd.DataFrame(matrix))
        else:
            display(pd.DataFrame(matrix).drop_duplicates().T)


display_overlap(df)

for name, group in df.groupby("test_name"):
    display_overlap(group, show_all=False, name=name)

All


,Absolute Diff,Percentage Diff,Count
brave Android 11 1.60.116 intent real_vs_brave Android 11 1.60.116 intent real,0,0.000000,10456
brave Android 11 1.60.116 intent real_vs_brave Ubuntu 22.04 1.59.120 (118.0.5993.88) selenium headless-new,269,2.572686,10456
brave Android 11 1.60.116 intent real_vs_brave Ubuntu 22.04 1.59.120 (118.0.5993.88) selenium xvfb,269,2.572686,10456
brave Android 11 1.60.116 intent real_vs_brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium headless-new,57,0.545142,10456
brave Android 11 1.60.116 intent real_vs_brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium xvfb,57,0.545142,10456
...,...,...,...
safari macOS 14.2.1 17.2 selenium real_vs_firefox macOS 14.2.1 120 selenium real,1104,10.558531,10456
safari macOS 14.2.1 17.2 selenium real_vs_firefox_beta Android 11 122.0b1 intent real,1406,13.446825,10456
safari macOS 14.2.1 17.2 selenium real_vs_opera Android 11 78.4.4143.75735 intent real,1910,18.267024,10456
safari macOS 14.2.1 17.2 selenium real_vs_operagx iPadOS 17.2 2.2.2 intent real,954,9.123948,10456


,brave Android 11 1.60.116 intent real,brave Ubuntu 22.04 1.59.120 (118.0.5993.88) selenium headless-new,brave Ubuntu 22.04 1.59.120 (118.0.5993.88) selenium xvfb,brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium headless-new,brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium xvfb,brave iPadOS 17.2 1.61 (23.12.18.22) intent real,brave macOS 14.2.1 1.60.118 (119.0.6045.163) selenium headless-new,brave macOS 14.2.1 1.60.118 (119.0.6045.163) selenium real,chrome Android 11 120.0.6099.44 intent real,chrome Ubuntu 22.04 119 selenium headless-new,chrome Ubuntu 22.04 119 selenium xvfb,chrome iPadOS 17.2 120.0.6099.119 intent real,chrome macOS 14.2.1 120 selenium headless-new,chrome macOS 14.2.1 120 selenium real,edge Ubuntu 22.04 119 selenium headless-new,edge Ubuntu 22.04 119 selenium xvfb,edge iPadOS 17.2 120.0.2210.116 intent real,edge macOS 14.2.1 120 selenium headless-new,edge macOS 14.2.1 120 selenium real,firefox Ubuntu 22.04 119 selenium headless,firefox Ubuntu 22.04 119 selenium xvfb,firefox iPadOS 17.2 121.0 (36782) intent real,firefox macOS 14.2.1 120 selenium headless,firefox macOS 14.2.1 120 selenium real,firefox_beta Android 11 122.0b1 intent real,opera Android 11 78.4.4143.75735 intent real,operagx iPadOS 17.2 2.2.2 intent real,safari macOS 14.2.1 17.2 selenium real
brave Android 11 1.60.116 intent real,0,269,269,57,57,2858,64,66,75,82,82,1967,83,83,190,191,1964,88,103,1039,1038,2545,1038,1033,1311,354,2547,2020
brave Ubuntu 22.04 1.59.120 (118.0.5993.88) selenium headless-new,269,0,0,236,236,2864,245,247,310,303,303,1994,306,306,333,334,1991,309,324,1066,1065,2572,1065,1060,1338,169,2557,2047
brave Ubuntu 22.04 1.59.120 (118.0.5993.88) selenium xvfb,269,0,0,236,236,2864,245,247,310,303,303,1994,306,306,333,334,1991,309,324,1066,1065,2572,1065,1060,1338,169,2557,2047
brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium headless-new,57,236,236,0,0,2864,9,11,74,67,67,1994,70,70,241,242,1991,73,88,1066,1065,2572,1065,1060,1338,405,2557,2047
brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium xvfb,57,236,236,0,0,2864,9,11,74,67,67,1994,70,70,241,242,1991,73,88,1066,1065,2572,1065,1060,1338,405,2557,2047
brave iPadOS 17.2 1.61 (23.12.18.22) intent real,2858,2864,2864,2864,2864,0,2872,2873,2858,2858,2858,1210,2860,2859,2858,2859,1203,2862,2847,2250,2249,1703,2249,2244,2491,2858,1641,1157
brave macOS 14.2.1 1.60.118 (119.0.6045.163) selenium headless-new,64,245,245,9,9,2872,0,18,81,76,76,2001,73,77,248,249,1998,82,97,1073,1072,2579,1072,1067,1345,412,2565,2054
brave macOS 14.2.1 1.60.118 (119.0.6045.163) selenium real,66,247,247,11,11,2873,18,0,83,78,78,2002,79,79,250,251,1999,82,99,1072,1071,2580,1071,1066,1343,414,2566,2055
chrome Android 11 120.0.6099.44 intent real,75,310,310,74,74,2858,81,83,0,7,7,1951,8,8,173,174,1948,13,28,1010,1009,2529,1009,1004,1283,337,2580,2004
chrome Ubuntu 22.04 119 selenium headless-new,82,303,303,67,67,2858,76,78,7,0,0,1958,3,3,180,181,1955,6,21,1017,1016,2536,1016,1011,1290,344,2579,2011


accesswindow_direct


,brave Android 11 1.60.116 intent real,brave Ubuntu 22.04 1.59.120 (118.0.5993.88) selenium headless-new,brave iPadOS 17.2 1.61 (23.12.18.22) intent real,chrome iPadOS 17.2 120.0.6099.119 intent real,firefox_beta Android 11 122.0b1 intent real
brave Android 11 1.60.116 intent real,0,4,176,24,24
brave Ubuntu 22.04 1.59.120 (118.0.5993.88) selenium headless-new,4,0,176,28,28
brave Ubuntu 22.04 1.59.120 (118.0.5993.88) selenium xvfb,4,0,176,28,28
brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium headless-new,4,0,176,28,28
brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium xvfb,4,0,176,28,28
brave iPadOS 17.2 1.61 (23.12.18.22) intent real,176,176,0,176,176
brave macOS 14.2.1 1.60.118 (119.0.6045.163) selenium headless-new,4,0,176,28,28
brave macOS 14.2.1 1.60.118 (119.0.6045.163) selenium real,4,0,176,28,28
chrome Android 11 120.0.6099.44 intent real,4,0,176,28,28
chrome Ubuntu 22.04 119 selenium headless-new,4,0,176,28,28


...

,brave Android 11 1.60.116 intent real,brave iPadOS 17.2 1.61 (23.12.18.22) intent real,firefox Ubuntu 22.04 119 selenium headless,firefox_beta Android 11 122.0b1 intent real
brave Android 11 1.60.116 intent real,0,176,32,40
brave Ubuntu 22.04 1.59.120 (118.0.5993.88) selenium headless-new,0,176,32,40
brave Ubuntu 22.04 1.59.120 (118.0.5993.88) selenium xvfb,0,176,32,40
brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium headless-new,0,176,32,40
brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium xvfb,0,176,32,40
brave iPadOS 17.2 1.61 (23.12.18.22) intent real,176,0,176,176
brave macOS 14.2.1 1.60.118 (119.0.6045.163) selenium headless-new,0,176,32,40
brave macOS 14.2.1 1.60.118 (119.0.6045.163) selenium real,0,176,32,40
chrome Android 11 120.0.6099.44 intent real,0,176,32,40
chrome Ubuntu 22.04 119 selenium headless-new,0,176,32,40


upgradeHSTS_direct


,brave Android 11 1.60.116 intent real,brave iPadOS 17.2 1.61 (23.12.18.22) intent real
brave Android 11 1.60.116 intent real,0,1
brave Ubuntu 22.04 1.59.120 (118.0.5993.88) selenium headless-new,0,1
brave Ubuntu 22.04 1.59.120 (118.0.5993.88) selenium xvfb,0,1
brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium headless-new,0,1
brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium xvfb,0,1
brave iPadOS 17.2 1.61 (23.12.18.22) intent real,1,0
brave macOS 14.2.1 1.60.118 (119.0.6045.163) selenium headless-new,0,1
brave macOS 14.2.1 1.60.118 (119.0.6045.163) selenium real,0,1
chrome Android 11 120.0.6099.44 intent real,0,1
chrome Ubuntu 22.04 119 selenium headless-new,0,1


upgradeHSTS_subdomain


,brave Android 11 1.60.116 intent real
brave Android 11 1.60.116 intent real,0
brave Ubuntu 22.04 1.59.120 (118.0.5993.88) selenium headless-new,0
brave Ubuntu 22.04 1.59.120 (118.0.5993.88) selenium xvfb,0
brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium headless-new,0
brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium xvfb,0
brave iPadOS 17.2 1.61 (23.12.18.22) intent real,0
brave macOS 14.2.1 1.60.118 (119.0.6045.163) selenium headless-new,0
brave macOS 14.2.1 1.60.118 (119.0.6045.163) selenium real,0
chrome Android 11 120.0.6099.44 intent real,0
chrome Ubuntu 22.04 119 selenium headless-new,0


### Clustering/Decision trees
- Try decision trees again!
- One tree for each test_name-relation
    - Limit on responses that have more than one outcome!
    - This will remove all the branches that are the same in all browsers (and would make trees large and confusing)
    - outcome = outcome_str (mostly binary, sometimes more options)
    - variables:
        - browser (includes os, automation mode, headless mode, version)
        - org_orgin = org_scheme + org_host
        - resp_origin = resp_scheme + resp_host
        - raw_headers
        - status_code
- H2O setup:
    - `sudo apt install openjdk-17-jre-headless graphviz`
    - `wget https://h2o-release.s3.amazonaws.com/h2o/rel-3.44.0/2/h2o-3.44.0.2.zip`
    - `unzip h2o-3.44.0.2.zip`
    - `cd h2o-3.44.0.2` and `java -jar h2o.jar`

In [516]:
# How many rows exist for tree creation
df[["test_name", "label", "relation_info"]].value_counts()

test_name                      label       relation_info 
fullscreen_iframe              PP          child_allow       10744
                                           child             10744
                                           direct            10744
                                           child_sandbox     10744
framing_iframe                 CSP-FA      nested             7616
                                           sandbox            7616
framing_object                 CSP-FA      sandbox            7616
                                           direct             7616
                                           nested             7616
framing_iframe                 CSP-FA      direct             7616
framing_embed                  CSP-FA      sandbox            7616
                                           nested             7616
script_execution_iframe        CSP-SCRIPT  sandbox            7616
                                           direct             7616
fram

In [23]:
h2o.init(nthreads=50, max_mem_size="100G", log_level="WARN")
h2o.no_progress()


# Limit on responses that have more than one outcome!
# This will remove all the branches that are the same in all browsers (and make trees large and confusing)
condition = df.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"])["outcome_str"].transform("nunique") != 1
tree_df = df.loc[condition]
print("Remove tests that are the same in all browsers", len(tree_df))
base_dir = f"trees/{datetime.today().strftime('%Y-%m-%dT%H:%M')}" 

for group_name, group in tree_df.groupby(["test_name", "relation_info"]):
    print(group_name)
    pred_cols = ["browser", "org_origin", "resp_origin", "status_code", "raw_header"]
    group = group[["outcome_str", *pred_cols]]
    tree = make_tree(group, pred_cols, group_name, base_dir)
    # print(tree)


Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "17.0.9" 2023-10-17; OpenJDK Runtime Environment (build 17.0.9+9-Ubuntu-122.04); OpenJDK 64-Bit Server VM (build 17.0.9+9-Ubuntu-122.04, mixed mode, sharing)
  Starting server from /home/ubuntu/.cache/pypoetry/virtualenvs/hp-OMYWa0k7-py3.11/lib/python3.11/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpqho6al4s
  JVM stdout: /tmp/tmpqho6al4s/h2o_ubuntu_started_from_python.out
  JVM stderr: /tmp/tmpqho6al4s/h2o_ubuntu_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,2 months and 2 days
H2O_cluster_name:,H2O_from_python_ubuntu_jovqdg
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,100 Gb
H2O_cluster_total_cores:,160
H2O_cluster_allowed_cores:,50
H2O_cluster_status:,"locked, healthy"


Remove tests that are the same in all browsers 82164


ZeroDivisionError: division by zero